In [ ]:
import asyncio
import json
import os
from bleak import BleakClient, BleakScanner

TARGET_NAME = "UWB_Tag" 

SERVICE_UUID = "service-uid"
CMD_CHAR_UUID = "char-cmd-uid"
LOGS_CHAR_UUID = "char-logs-uid"

SAVE_FOLDER = "logs"  
NOTIFY_TIMEOUT = 5  

os.makedirs(SAVE_FOLDER, exist_ok=True)

logs_data = ""

In [ ]:
def notification_handler(sender, data):
    global logs_data
    logs_data += data.decode('utf-8')
    print(data.decode('utf-8'), end='')

async def main():
    print("Scanning for BLE devices...")
    devices = await BleakScanner.discover()
    target = None
    for d in devices:
        if TARGET_NAME == str(d.address) or TARGET_NAME == str(d.name):
            target = d
            break

    if target is None:
        raise FileNotFoundError(f"Device '{TARGET_NAME}' not found!")

    print(f"Found {TARGET_NAME} at {target.address}. Connecting...")
    
    async with BleakClient(target.address) as client:
        if not client.is_connected:
            raise ValueError("Failed to connect.")

        await client.start_notify(LOGS_CHAR_UUID, notification_handler)
        await asyncio.sleep(0.5) 
        print("\nRequesting logs...")
        await client.write_gatt_char(CMD_CHAR_UUID, b"GET_LOGS\n", response=False)
        await asyncio.sleep(NOTIFY_TIMEOUT)
        await client.stop_notify(LOGS_CHAR_UUID)
        print("\nFinished receiving logs.")

    logs_entries = logs_data.strip().split("\n")

    for i, entry in enumerate(logs_entries):
        try:
            data = json.loads(entry)
        except json.JSONDecodeError:
            print(f"invalid JSON: {entry}")
            continue

        filename = os.path.join(SAVE_FOLDER, f"log_{i+1}.json")
        with open(filename, "w") as f:
            json.dump(data, f, indent=4)

In [17]:
await main()

Scanning for BLE devices...
D7:6B:D0:57:CB:20
None
C1:B3:BD:05:B3:C7
None
78:3F:ED:78:73:9A
None
0A:23:CB:53:81:41
None
69:67:33:53:06:D2
None
5A:35:A6:F6:59:43
None
D0:EF:76:35:84:F2
UWB_Tag
Found UWB_Tag at D0:EF:76:35:84:F2. Connecting...
Connected!

Requesting logs...
{"T":721030,"entries":[{"T":721025,"A":"A49C","R":2.35,"Rx":-52.27}]}

{"T":841037,"entries":[{"T":841032,"A":"A49C","R":2.28,"Rx":-50.74}]}

{"T":241001,"entries":[{"T":240998,"A":"A49C","R":2.17,"Rx":-52.95}]}

{"T":361010,"entries":[{"T":361005,"A":"A49C","R":2.12,"Rx":-52.17}]}

{"T":481018,"entries":[{"T":481013,"A":"A49C","R":2.04,"Rx":-54.53}]}

{"T":601026,"entries":[{"T":601021,"A":"A49C","R":2.27,"Rx":-51.86}]}

{"T":120993,"entries":[{"T":120988,"A":"A49C","R":2.14,"Rx":-51.01}]}


Finished receiving logs.
Saving 13 logs...
Skipping invalid JSON: 
Skipping invalid JSON: 
Skipping invalid JSON: 
Skipping invalid JSON: 
Skipping invalid JSON: 
Skipping invalid JSON: 
Logs saved in 'logs' folder.
